### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

### CONSTANTES

### QUERIES

In [45]:
q_t1 = '''WITH orders_table AS (
    SELECT o.user.id AS user,
           o.registered_date AS fecha,
           o.order_id AS order_id,
           bt.business_type_name AS business,
           CASE WHEN o.is_acquisition = 1 THEN TRUE ELSE FALSE END AS acq,
           cn.country_name AS country,
           IFNULL(tca.talon_campaign_name,'-') AS campaign
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o 
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_business_type` AS bt ON o.business_type_id = bt.business_type_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
    LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON tc.talon_campaign_id = tca.talon_campaign_id
    WHERE o.registered_date < CURRENT_DATE()
          AND o.order_status = 'CONFIRMED'),
    qc_table AS (
    SELECT ot.user AS user,
           ot.business AS business,
           ot.order_id AS order_id,
           ot.country AS country,
           ot.acq AS acq,
           ot.campaign AS campaign
    FROM orders_table AS ot
    WHERE ot.business NOT IN ('Restaurant','Coffee','Courier','Courier Business')
          AND ot.fecha BETWEEN DATE('2021-07-01') AND DATE('2021-07-31')),
    history_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           SUM(CASE WHEN ot.business NOT IN ('Restaurant','Coffee','Courier','Courier Business') THEN 1 ELSE 0 END) AS qc
    FROM orders_table AS ot
    WHERE ot.fecha < DATE('2021-07-01')
          OR ot.acq
    GROUP BY 1,2)
SELECT qct.country AS Country,
       qct.business AS Business,
       qct.campaign AS Campaign,
       COUNT(DISTINCT qct.user) AS Active_QC,
       COUNT(DISTINCT CASE WHEN qct.acq = FALSE AND (ht.qc = 0 OR ht.qc IS NULL) THEN qct.user ELSE NULL END) AS Adq_QC
FROM qc_table AS qct
INNER JOIN (SELECT user, country, MIN(order_id) AS orders
            FROM qc_table GROUP BY 1,2) AS qct_min ON qct.user = qct_min.user AND qct.country = qct_min.country AND qct.order_id = qct_min.orders
LEFT JOIN history_table AS ht ON qct.user = ht.user AND qct.country = ht.country
WHERE qct.country IN ('Argentina','Chile')
GROUP BY 1,2,3
ORDER BY 1'''

q_t2 = '''WITH orders_table AS (
    SELECT o.user.id AS user,
           o.registered_date AS fecha,
           o.order_id AS order_id,
           bt.business_type_name AS business,
           CASE WHEN o.is_acquisition = 1 THEN TRUE ELSE FALSE END AS acq,
           cn.country_name AS country,
           IFNULL(tca.talon_campaign_name,'-') AS campaign
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o 
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_business_type` AS bt ON o.business_type_id = bt.business_type_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
    LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON tc.talon_campaign_id = tca.talon_campaign_id
    WHERE o.registered_date < CURRENT_DATE()
          AND o.order_status = 'CONFIRMED'),
    qc_table AS (
    SELECT ot.user AS user,
           ot.business AS business,
           ot.order_id AS order_id,
           ot.country AS country,
           ot.acq AS acq,
           ot.campaign AS campaign
    FROM orders_table AS ot
    WHERE ot.business NOT IN ('Restaurant','Coffee','Courier','Courier Business')
          AND ot.fecha BETWEEN DATE('2021-06-01') AND DATE('2021-06-30')),
    history_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           SUM(CASE WHEN ot.business NOT IN ('Restaurant','Coffee','Courier','Courier Business') THEN 1 ELSE 0 END) AS qc
    FROM orders_table AS ot
    WHERE ot.fecha < DATE('2021-06-01')
          OR ot.acq
    GROUP BY 1,2)
SELECT qct.country AS Country,
       qct.business AS Business,
       qct.campaign AS Campaign,
       COUNT(DISTINCT qct.user) AS Active_QC,
       COUNT(DISTINCT CASE WHEN qct.acq = FALSE AND (ht.qc = 0 OR ht.qc IS NULL) THEN qct.user ELSE NULL END) AS Adq_QC
FROM qc_table AS qct
INNER JOIN (SELECT user, country, MIN(order_id) AS orders
            FROM qc_table GROUP BY 1,2) AS qct_min ON qct.user = qct_min.user AND qct.country = qct_min.country AND qct.order_id = qct_min.orders
LEFT JOIN history_table AS ht ON qct.user = ht.user AND qct.country = ht.country
WHERE qct.country IN ('Argentina','Chile')
GROUP BY 1,2,3
ORDER BY 1'''

In [46]:
# Descargo la data
bq_t1 = pd.io.gbq.read_gbq(q_t1, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 1505/1505 [00:00<00:00, 3140.40rows/s]


In [47]:
bq_t2 = pd.io.gbq.read_gbq(q_t2, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 1626/1626 [00:00<00:00, 3856.15rows/s]


In [79]:
# Copio las bases
t1 = bq_t1.copy()
t2 = bq_t2.copy()

### TRABAJO

In [80]:
# Trabajo Trials 1
val = [i for i in t1.columns if i not in ['Country','Business','Campaign']]
t1[val] = t1[val].astype(int)
col = ['Country','Business','Campaign']
t1[col] = t1[col].apply(lambda x: x.astype(str).str.upper())
t1['Campaign'] = t1['Campaign'].str.replace(' ', '')
# Trabajo Trials 2
val = [i for i in t2.columns if i not in ['Country','Business','Campaign']]
t2[val] = t2[val].astype(int)
col = ['Country','Business','Campaign']
t2[col] = t2[col].apply(lambda x: x.astype(str).str.upper())
t2['Campaign'] = t2['Campaign'].str.replace(' ', '')

In [81]:
# Cambio el nombre a las columnas
cols = ['Active_QC','Adq_QC']
t1.columns = [str(x)+'-2021/07' if x in cols else x for x in t1.columns ]
t2.columns = [str(x)+'-2021/06' if x in cols else x for x in t2.columns ]

In [89]:
# Hago un merge
on = ['Country','Business','Campaign']
final = t1.merge(t2,on=on,how='outer')
final.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [110]:
# Dropeo las columnas que no necesito
cols = [x for x in final.columns if 'Active_QC' in x]
final.drop(columns=cols,inplace=True)

### CARGA

In [113]:
carga(final,'1vapwjCqVweQxL2_Auff4R-99umM95R3eXm20DrnFwnA','Crudo')

Carga Correcta!
